In [5]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [1]:
source_text = "llm.txt"

with open(source_text, 'r') as f:
    text = f.read()

CHUNK_SIZE = 1000
chunked_text = [text[i: i+CHUNK_SIZE] for i in range(0, len(text), CHUNK_SIZE)]

In [2]:
vector_store_path = "hub://secufibre/space_exploration_v1"

In [4]:
from deeplake.core.vectorstore.deeplake_vectorstore import VectorStore
import deeplake.util

try:
    vector_store = VectorStore(path=vector_store_path)
    print("Vector store exists")
except FileNotFoundError:
    print("Vector store does not exist. You can create it.")

Your Deep Lake dataset has been successfully created!


Vector store exists


In [7]:
import openai

def embedding_function(texts, model="text-embedding-3-small"):
    if isinstance(texts, str):
        texts = [texts]
    texts = [t.replace("\n", " ") for t in texts]
    return [data.embedding for data in openai.embeddings.create(input=texts, model=model).data]

In [8]:
add_to_vector_store = True
if add_to_vector_store == True:
    with open(source_text, 'r') as f:
        text = f.read()
        CHUNK_SIZE = 1000
        chunked_text = [text[i: i+CHUNK_SIZE] for i in range(0, len(text), CHUNK_SIZE)]

vector_store.add(
    text=chunked_text,
    embedding_function=embedding_function,
    embedding_data=chunked_text,
    metadata=[{"source": source_text}]*len(chunked_text)
)

Creating 1654 embeddings in 4 batches of size 500:: 100%|██████████| 4/4 [01:22<00:00, 20.69s/it]

Dataset(path='hub://secufibre/space_exploration_v1', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype       shape       dtype  compression
  -------    -------     -------     -------  ------- 
   text       text      (1654, 1)      str     None   
 metadata     json      (1654, 1)      str     None   
 embedding  embedding  (1654, 1536)  float32   None   
    id        text      (1654, 1)      str     None   


In [9]:
vector_store.summary()

Dataset(path='hub://secufibre/space_exploration_v1', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype       shape       dtype  compression
  -------    -------     -------     -------  ------- 
   text       text      (1654, 1)      str     None   
 metadata     json      (1654, 1)      str     None   
 embedding  embedding  (1654, 1536)  float32   None   
    id        text      (1654, 1)      str     None   


In [10]:
ds = deeplake.load(vector_store_path)

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/secufibre/space_exploration_v1



hub://secufibre/space_exploration_v1 loaded successfully.



In [11]:
ds_size = ds.size_approx()

In [12]:
ds_size_mb = ds_size / 1024 ** 2
print(f"Dataset size in megabytes: {ds_size_mb:.5f} MB")

ds_size_gb = ds_size / 1024 ** 3
print(f"Dataset size in gigabytes: {ds_size_gb:.5f} GB")

Dataset size in megabytes: 55.31311 MB
Dataset size in gigabytes: 0.05402 GB
